# 股吧情绪分析

目标地址

http://guba.eastmoney.com/list,of161725.html


代理池

https://github.com/jhao104/proxy_pool

## 一、导入程序

In [2]:
# import csv
import time
import random
import requests
import traceback
from time import sleep
from fake_useragent import UserAgent
from lxml import etree
import pandas as pd
import re
import math


## 二、程序设计

### 1、数据源获取

> 无限请求，直到请求到数据

In [3]:
# page = 1  #设置爬取的页数
# fundcode = 161725  #可替换任意基金代码
# sleep(random.uniform(1, 2))  #随机出现1-2之间的数，包含小数
# headers = {"User-Agent": UserAgent(verify_ssl=False).random}
# url = f'http://guba.eastmoney.com/list,of{fundcode}_{page}.html'
# response = requests.get(url, headers=headers, timeout=10)
# print(response)


def get_fund_data(fundcode='161725', page=1, proxies=None):
    try:
        # page = 1  #设置爬取的页数
        # fundcode = 161725  #可替换任意基金代码
        sleep(random.uniform(1, 5))  #随机出现1-2之间的数，包含小数
        headers = {"User-Agent": UserAgent().random}
        url = f'http://guba.eastmoney.com/list,of{fundcode}_{page}.html'
        print('开始请求')
        print(url)
        response = requests.get(url, headers=headers, timeout=10, proxies=proxies)
        # print(response)
        return response.text
    except requests.exceptions.ConnectionError:
        t = random.uniform(1, 5)
        print(f'ConnectionError -- please wait {t} seconds')
        time.sleep(random.uniform(1, 5))
        return get_fund_data(fundcode, page, proxies)
    except requests.exceptions.ChunkedEncodingError:
        t = random.uniform(1, 5)
        print(f'ChunkedEncodingError -- please wait {t} seconds')
        time.sleep(t)
        return get_fund_data(fundcode, page, proxies)
    except:
        t = random.uniform(1, 5)
        print(
            f'Unfortunitely -- An Unknow Error Happened, Please wait {t} seconds'
        )
        time.sleep(t)
        return get_fund_data(fundcode, page, proxies)

response = get_fund_data()

开始请求
http://guba.eastmoney.com/list,of161725_1.html


### 2、解析数据

In [4]:
# 获取总页数
def parse_total(html):
    parse = etree.HTML(html)
    # print(etree.tostring(parse))
    div = parse.xpath('//*[@id="articlelistnew"]/div')[-2]
    total = div.text
    # 总共篇数
    total = re.findall(r'(\d+)', total)[0]
    # 总页数, 总篇数/单页条数，再向上取整
    total = math.ceil((int(total) / 80))
    return total

total = parse_total(response)
# etree.tostring(total)
total


9529

In [5]:
# 获取数据源
def parse_fund(html):
    parse = etree.HTML(html)  # 解析网页
    items = parse.xpath('//*[@id="articlelistnew"]/div')[1:-2]
    data = []
    columns = ['阅读', '评论', '标题', '作者', '时间']
    for item in items:
        _arr = [
            ''.join(item.xpath('./span[1]/text()')).strip(),
            ''.join(item.xpath('./span[2]/text()')).strip(),
            ''.join(item.xpath('./span[3]/a/text()')).strip(),
            ''.join(item.xpath('./span[4]/a/font/text()')).strip(),
            ''.join(item.xpath('./span[5]/text()')).strip()
        ]
        # item = {
        #     '阅读': ''.join(item.xpath('./span[1]/text()')).strip(),
        #     '评论': ''.join(item.xpath('./span[2]/text()')).strip(),
        #     '标题': ''.join(item.xpath('./span[3]/a/text()')).strip(),
        #     '作者': ''.join(item.xpath('./span[4]/a/font/text()')).strip(),
        #     '时间': ''.join(item.xpath('./span[5]/text()')).strip()
        # }
        data.append(_arr)
        # print(item)
    # print(arr)
    df = pd.DataFrame(data=data,columns=columns)
    return df
df = parse_fund(response)
df

,阅读,评论,标题,作者,时间
0,1089,1,【有机会瓜分66666积分】这个中秋，与德邦共赴“飞花,德邦基金,09-10 19:07
1,1270,11,有奖聊 | 那些年我们一起用过的电池,嘉实基金,09-10 19:50
2,11269,20,【有机会瓜分3000壕礼】神仙智能家电大种草！,富国基金,09-10 18:54
3,3787,69,【3000元京东卡】全能成长猎手钟帅重磅新基来袭！记得,华夏基金,09-10 19:17
4,2100,30,有奖互动 | 月圆邀桃“礼” 富达递师恩,富达国际,09-10 17:41
...,...,...,...,...,...
80,2512,3,医药，互联网，新能源传利好，白酒利空，抄底该如何选,大眼海豚,09-10 20:09
81,1172,2,下周走势研究分析，煤炭看量能，白酒博弈中秋行情。,星期五的大平,09-10 20:09
82,244,1,今年一共就涨两个月，到目前为止整整跌了七个月,ckiiii,09-10 20:09
83,10509,20,银行、证券、白酒、医药、新能源、半导体板块走势技术,野庄炖基,09-10 20:09


### 3、保存到csv

In [6]:
df.to_csv('股吧情绪分析.csv', index=False)

## 三、设计主函数

### 1、添加代理池

参考：https://github.com/jhao104/proxy_pool

In [7]:

def get_proxy():
    return requests.get("http://demo.spiderpy.cn/get/").json()


def delete_proxy(proxy):
    requests.get(f"http://demo.spiderpy.cn/delete/?proxy={proxy}")

### 2、设计分页异常函数

In [10]:
proxyObj = get_proxy()
proxy = proxyObj.get("proxy")
region = proxyObj.get('region')
proxies = { "http": f"http://{proxy}" }
print(f'代理:{proxy}, 位置:{region}')

def main(fundcode='161725', page = 1, total=None):
    # try:
    html = get_fund_data(fundcode, page, proxies)
    df = pd.DataFrame()
    if total is None:
        total = parse_total(html)
    print(f'第{page}页/总页数: {total}')
    for page_i in range(page, total):
        # 一定要加随机时间，防止被反爬
        # time.sleep(random.uniform(1, 5))
        if page != 1:
            html = get_fund_data(fundcode, page_i, proxies)
            df = parse_fund(html)
        df = parse_fund(html)
        # 数据为空的情况
        if df.empty is True:
            # 数据为空，重新执行
            main(fundcode, page, total)
        print(f'获取第{page_i}页数据内容')
        print(df)
        df.to_csv('./股吧情绪分析.csv', mode='a', index=False, header=False)
        print(f'第{page_i}页数据内容保存成功!')
    # except Exception as e:
    #     print(e)
    #     print(f'第{page}页有异常，正在重新请求..')
    # main(fundcode, page, total)
main()

代理:20.187.70.249:8000, 位置:中国 香港  microsoft.com
